# Anomaly WML Deployment Noteook

This notebook demonstrates the use of AI Model Factory recipes to deploy energy loss pipeline to WML

Last Updated November 29 2023

## Import libraries

In [1]:
import json
import requests
import yaml

## Set Variables

model_uri - URI to fetch the model from \
deployment_name - name to set for the deployment

In [2]:
model_factory_config_file_name = "../config/model_info.yml"


with open(model_factory_config_file_name, 'r') as file:
    model_factory_config = yaml.safe_load(file)

print(model_factory_config)

{'model_uri': 'runs:/ec9217cf80f841f5a3bebca8ef8632be/energyloss_AD', 's3_uri': 's3://testdataupload/16/ec9217cf80f841f5a3bebca8ef8632be/artifacts/energyloss_AD', 'train_job_id': '1eecb3db-4b03-49b7-a781-7c1b2e12a707'}


In [3]:
model_uri = model_factory_config['s3_uri']
deployment_name = "energyloss_AD_111723_joe"

In [4]:
model_uri

's3://testdataupload/16/ec9217cf80f841f5a3bebca8ef8632be/artifacts/energyloss_AD'

In [5]:
print(model_uri)

s3://testdataupload/16/ec9217cf80f841f5a3bebca8ef8632be/artifacts/energyloss_AD


In [6]:
endpoint_url = "http://localhost:8000/ibm/modelfactory/service"
credentials_path = "../../config/credentials/wml-credentials.json"

In [7]:
url = endpoint_url + "/deployment/wml/create"

## Create a request to call WML deployment recipe

In [8]:
payload = {
    'endpoint': 'electrical_transformer_deployments',
    'deployment_name': deployment_name,
    'model_uri': model_uri,
    'flavor': 'sklearn',
    'software_spec_name': 'modelfactory',
    
}

In [9]:
files={
      "wml_credentials_file": ('modelfactory-modelfactory-wml-credentials.json',open(credentials_path,'rb'),'application/json')
}

In [10]:
headers = {
  'accept': 'application/json'
}

In [11]:
response = requests.request("POST", url, headers=headers, data=payload, files=files)

In [12]:
print(response.text)

{"job_id":"08bf718e-0802-4870-bca6-a82261f38302","message":"WML deployment energyloss_AD_111723_joe submitted for creation"}


## Check the status of the job

In [13]:
job_id = json.loads(response.text)["job_id"]

In [14]:
url = endpoint_url + "/log" + f"/{job_id}"

In [15]:
response = requests.request("GET", url, headers=headers, data={}, files=[])

In [16]:
print(json.loads(response.text)["logs"])

recipe_file_path /tmp/ray/session_2023-11-17_16-39-41_914185_1/runtime_resources/working_dir_files/_ray_pkg_d788be949e2811fd/Recipe.create
entrypoint_path /tmp/ray/session_2023-11-17_16-39-41_914185_1/runtime_resources/working_dir_files/_ray_pkg_d788be949e2811fd/main.py



## Get the summary of the job

In [17]:
url = endpoint_url + "/summary" + f"/{job_id}"

In [18]:
response = requests.request("GET", url, headers=headers, data={}, files=[])

In [19]:
# More print functions using display 
from IPython.display import display, HTML
import requests

def print_job_details(job_id, endpoint_url):
    # Extract the job ID and construct the URL
    url = endpoint_url + "/summary/" + job_id
    
    # Send a GET request to fetch the job status
    get_response = requests.get(url, headers={})
    summary_data = get_response.json()
    
    # Display the job status
    display(HTML(print_keys_and_values(summary_data)))

    
def print_keys_and_values(json_data):
    # Start the HTML code
    html_code = "<div style='font-family: Arial; font-size: 1.2em;'>"
    
    # Add the job details to the HTML code
    html_code += f"<p>job_id: {json_data['job_id']}</p>"
    if "status" in json_data:
        html_code += f"<p>status: {json_data['status']}</p>"
    html_code += "<br>"
    if "detailed_summary" in json_data:
        for summary in json_data['detailed_summary']:
            html_code += f"<p>run_id: {summary['run_id']}</p>"
            html_code += f"<p>experiment_id: {summary['experiment_id']}</p>"
            html_code += f"<p>status: {summary['status']}</p>"
            html_code += f"<p>artifact_uri: {summary['artifact_uri']}</p>"
            html_code += f"<p>artifact_name: {summary.get('tags.artifact_name', 'No artifact_name found')}</p>"
            html_code += "<br>"
    
    # Close the HTML code
    html_code += "</div>"
    
    return html_code

In [23]:
print_job_details(job_id, endpoint_url)

### Saving details for deployment

In [28]:
json_data = print_job_details(job_id,endpoint_url)
json_data

In [29]:
# Check the summary
summary_url = endpoint_url + "/summary/"
get_response = requests.get(summary_url+job_id, headers={})
summary_data = get_response.json()
print(summary_data)


{'job_id': '08bf718e-0802-4870-bca6-a82261f38302', 'status': 'DONE', 'summary': {'deployment_details': {'entity': {'asset': {'id': 'f97c333a-f6c7-4da2-a891-3a45fc641c69', 'rev': '1'}, 'custom': {'AWS_ACCESS_KEY_ID': 'e6d28d7de535464da066c608726b4ee3', 'AWS_SECRET_ACCESS_KEY': '2fc6029cc5d799b6c572c0e730d6bc912c047cb1dddce409', 'MLFLOW_S3_ENDPOINT_URL': 'https://s3.us-south.cloud-object-storage.appdomain.cloud', 'MLFLOW_TRACKING_URI': 'http://mlflow:5000'}, 'deployed_asset_type': 'model', 'hardware_spec': {'id': 'b128f957-581d-46d0-95b6-8af5cd5be580', 'name': 'XXS', 'num_nodes': 1}, 'name': 'energyloss_AD_111723_joe', 'online': {}, 'space_id': 'a0521ce5-5547-41e2-8495-66012d7e8780', 'status': {'online_url': {'url': 'https://cpd-ibm-cpd.modelfactory-9ca4d14d48413d18ce61b80811ba4308-0000.us-south.containers.appdomain.cloud/ml/v4/deployments/654f68d2-d8e6-43b4-a13b-0ec2f265bb93/predictions'}, 'serving_urls': ['https://cpd-ibm-cpd.modelfactory-9ca4d14d48413d18ce61b80811ba4308-0000.us-south.

In [30]:
summary_data

{'job_id': '08bf718e-0802-4870-bca6-a82261f38302',
 'status': 'DONE',
 'summary': {'deployment_details': {'entity': {'asset': {'id': 'f97c333a-f6c7-4da2-a891-3a45fc641c69',
     'rev': '1'},
    'custom': {'AWS_ACCESS_KEY_ID': 'e6d28d7de535464da066c608726b4ee3',
     'AWS_SECRET_ACCESS_KEY': '2fc6029cc5d799b6c572c0e730d6bc912c047cb1dddce409',
     'MLFLOW_S3_ENDPOINT_URL': 'https://s3.us-south.cloud-object-storage.appdomain.cloud',
     'MLFLOW_TRACKING_URI': 'http://mlflow:5000'},
    'deployed_asset_type': 'model',
    'hardware_spec': {'id': 'b128f957-581d-46d0-95b6-8af5cd5be580',
     'name': 'XXS',
     'num_nodes': 1},
    'name': 'energyloss_AD_111723_joe',
    'online': {},
    'space_id': 'a0521ce5-5547-41e2-8495-66012d7e8780',
    'status': {'online_url': {'url': 'https://cpd-ibm-cpd.modelfactory-9ca4d14d48413d18ce61b80811ba4308-0000.us-south.containers.appdomain.cloud/ml/v4/deployments/654f68d2-d8e6-43b4-a13b-0ec2f265bb93/predictions'},
     'serving_urls': ['https://cpd-ibm

In [31]:
deployment_endpoint = summary_data['summary']['deployment_details']['entity']['status']

In [32]:
from datetime import datetime

deployment_endpoint['cp4d_url'] = deployment_endpoint['serving_urls'][0]
# Get the current date
current_date = datetime.now()

# Format the date and create the string
version_string = f"?version={current_date.strftime('%Y-%m-%d')}"
deployment_endpoint['cp4d_url'] = deployment_endpoint['cp4d_url'] + version_string

In [33]:
deployment_endpoint

{'online_url': {'url': 'https://cpd-ibm-cpd.modelfactory-9ca4d14d48413d18ce61b80811ba4308-0000.us-south.containers.appdomain.cloud/ml/v4/deployments/654f68d2-d8e6-43b4-a13b-0ec2f265bb93/predictions'},
 'serving_urls': ['https://cpd-ibm-cpd.modelfactory-9ca4d14d48413d18ce61b80811ba4308-0000.us-south.containers.appdomain.cloud/ml/v4/deployments/654f68d2-d8e6-43b4-a13b-0ec2f265bb93/predictions'],
 'state': 'ready',
 'cp4d_url': 'https://cpd-ibm-cpd.modelfactory-9ca4d14d48413d18ce61b80811ba4308-0000.us-south.containers.appdomain.cloud/ml/v4/deployments/654f68d2-d8e6-43b4-a13b-0ec2f265bb93/predictions?version=2023-11-17'}

In [34]:
output_config_file = 'deployment_info.yml'

In [35]:
with open("../config/" + output_config_file,"w") as file:
    yaml.dump(deployment_endpoint, file)